# IT Incident Category Prediction

This solution, uses textual description of incident from the ticket data to predict its category. It uses an attention based pretrained model to  classify the tickets over a predefined set of incident categories using these textual descriptions. These incident categories along with the respective tickets are provided as output which can be used by an IT service management tool. This reduces the efforts required by L0 associates to read through the description and assign the categories.

This sample notebook shows you how to deploy IT Incident Category Prediction using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to IT Incident Category Prediction. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output result](#D.-Output-result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page **IT Incident Category Prediction**
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/mphasis-marketplace-incident-category-prediction'

In [2]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
#from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
#import urllib.request
import numpy as np

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/OpenSSL/crypto.py:12: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography import x509
No handlers could be found for logger "sagemaker"


In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name='incident-category-prediction'

content_type='text/csv'

real_time_inference_instance_type='ml.m5.xlarge'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:

def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type=content_type)

#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [6]:
import pandas as pd
file_name = './data/input/Input-2.csv'


<Add code snippet that shows the payload contents>

In [7]:
output_file_name = 'output-2.csv'

#### C. Perform real-time inference

In [8]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name 'incident-category-prediction' \
    --body fileb://$file_name \
    --content-type 'text/csv' \
    --region us-east-2 \
    output-2.csv

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/OpenSSL/crypto.py:12: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography import x509
{
    "InvokedProductionVariant": "AllTraffic", 
    "ContentType": "text/csv; charset=utf-8"
}


#### D. Output result

In [9]:
df = pd.read_csv("output-2.csv")
#df  = df.drop('Unnamed: 0',1)
df.head(10)

,Unnamed: 0,Incident_number,Description,Topic_number,Topic_name
0,0,1,hi since recruiter lead permission approve req...,4,Generic Assistance
1,1,7,change role hello dear please change pm role s...,4,Generic Assistance
2,2,8,hello received workstation granted ad please h...,4,Generic Assistance
3,3,9,area hello please card area floor thank kind r...,4,Generic Assistance
4,4,10,badge lost hello lost badge need one thank you,4,Generic Assistance
5,5,11,sent stolen phone hi please raise block card f...,4,Generic Assistance
6,6,12,sent friday need re start importance high hi s...,4,Generic Assistance
7,7,13,sent friday december restart hello can you ple...,4,Generic Assistance
8,8,14,sent wednesday unable remove reservation hi pl...,4,Generic Assistance
9,9,15,phone gives error parse error when trying dire...,4,Generic Assistance


#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [10]:
predictor=sage.RealTimePredictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [11]:
#upload the batch-transform job input files to S3
transform_input_folder = "./data/input/Input-2.csv"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/incident-category-prediction/Input-2.csv


In [12]:
#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

.......................... * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [11/Nov/2021 11:27:47] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [11/Nov/2021 11:27:47] "#033GET /execution-parameters HTTP/1.1#033" 404 -
169.254.255.130 - - [11/Nov/2021 11:27:47] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [11/Nov/2021 11:27:47] "#033GET /execution-parameters HTTP/1.1#033" 404 -
#015  0%|          | 0/1 [00:00<?, ?it/s]#015100%|##########| 1/1 [00:00<00:00, 1501.18it/s]
2021-11-11T11:27:47.939:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
#015  0%|          | 0/1 [00:00<?, ?it/s]#015

In [13]:
#output is available on following path
transformer.output_path

's3://sagemaker-us-east-2-786796469737/mphasis-marketplace-incident-category-p-2021-11-11-11-23-40-361'

In [14]:
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('./data/output/output-2.csv', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, 'mphasis-marketplace-incident-category-p-2021-11-11-11-23-40-361'+'/Input-2.csv.out', f)
    print("Output file loaded from bucket")

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/boto3/compat.py:86: PythonDeprecationWarning: Boto3 will no longer support Python 2.7 starting July 15, 2021. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.6 or later. More information can be found here: https://aws.amazon.com/blogs/developer/announcing-end-of-support-for-python-2-7-in-aws-sdk-for-python-and-aws-cli-v1/
  warnings.warn(warning, PythonDeprecationWarning)


Output file loaded from bucket


In [15]:
df = pd.read_csv("./data/output/output-2.csv")
#df  = df.drop('Unnamed: 0',1)
df.head(10)

,Unnamed: 0,Incident_number,Description,Topic_number,Topic_name
0,0,1,hi since recruiter lead permission approve req...,4,Generic Assistance
1,1,7,change role hello dear please change pm role s...,4,Generic Assistance
2,2,8,hello received workstation granted ad please h...,4,Generic Assistance
3,3,9,area hello please card area floor thank kind r...,4,Generic Assistance
4,4,10,badge lost hello lost badge need one thank you,4,Generic Assistance
5,5,11,sent stolen phone hi please raise block card f...,4,Generic Assistance
6,6,12,sent friday need re start importance high hi s...,4,Generic Assistance
7,7,13,sent friday december restart hello can you ple...,4,Generic Assistance
8,8,14,sent wednesday unable remove reservation hi pl...,4,Generic Assistance
9,9,15,phone gives error parse error when trying dire...,4,Generic Assistance


### 4. Clean-up

#### A. Delete the model

In [16]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

